In [ ]:
import os
import sys
import numpy as np
import pandas as pd
import xarray as xr
from natsort import natsorted
from os.path import join as pjoin
from scipy.stats import zscore
from dask.distributed import Client, LocalCluster

sys.path.append('../../')
import circletrack_neural as ctn

project_dir = 'MultiCon_Imaging'
experiment_dir = 'MultiCon_Imaging5'
mouse_list = ['mc44', 'mc46', 'mc49', 'mc51', 'mc52']
mouse_list = ['mc52'] ## must do mc51 still
dpath = f'../../../{project_dir}/{experiment_dir}/output/aligned_minian/'
spath = f'../../../{project_dir}/{experiment_dir}/output/reward_modulation/'
data_of_interest = 'aligned_minian'
data_type = 'C'
starting_idx = 0
window_size = 90 ## in frames
sampling_rate = 1/30
num_simulations = 500
percentiles = (5, 95)
correct_size = (window_size * 2) + 1 ## since inclusive on both ends

xr.set_options(keep_attrs=True)

## Set seed
np.random.seed(24601)

## Create cluster
cluster = LocalCluster(n_workers=16,
                       memory_limit='6GB',
                       resources={'MEM': 1},
                       threads_per_worker=2,
                       dashboard_address=':8787')
client = Client(cluster)

## Loop through mouse list
for mouse in mouse_list:
    mpath = pjoin(dpath, f'{mouse}/{data_type}')
    for idx, session in enumerate(natsorted(os.listdir(mpath))):
        if idx < starting_idx:
            pass 
        else:
            print(session)
            save_path = pjoin(spath, f'{mouse}/{data_type}')
            C = xr.open_dataset(pjoin(mpath, session))[data_type]
            num_neurons = C.shape[0]
            ## Normalize data
            zdata = xr.apply_ufunc(
                    zscore,
                    C.chunk({'frame': -1, 'unit_id': 50}),
                    input_core_dims=[['frame']],
                    output_core_dims=[['frame']],
                    kwargs={'axis': 1},
                    dask='parallelized'
            ).compute()
            ## Compute actual reward-related activity
            windowed_data, windowed_sem, rw_pre, rw_post, rw_one_array, rw_two_array = ctn.reward_activity(zdata, 
                                                                                                            correct_size=correct_size, 
                                                                                                            window_size=window_size,
                                                                                                            reward_one=zdata.attrs['reward_one'],
                                                                                                            reward_two=zdata.attrs['reward_two'])
            ## Compute shuffle distribution
            df = ctn.compute_shuffled_rw_diff(zdata, 
                                            correct_size=correct_size, 
                                            window_size=window_size, 
                                            num_simulations=num_simulations)
            ## Get difference in observed activity around reward locations
            ## and compare observed activity to a shuffle distribution
            rw_one_diff = rw_post[:, 0] - rw_pre[:, 0]
            rw_two_diff = rw_post[:, 1] - rw_pre[:, 1]
            rw_one_pos = np.array([rw_one_diff[neuron] > np.percentile(df['difference'][(df['unit'] == neuron) & (df['dim'] == 0)], percentiles[1]) for neuron in np.arange(0, num_neurons)])
            rw_two_pos = np.array([rw_two_diff[neuron] > np.percentile(df['difference'][(df['unit'] == neuron) & (df['dim'] == 1)], percentiles[1]) for neuron in np.arange(0, num_neurons)])
            rw_one_neg = np.array([rw_one_diff[neuron] < np.percentile(df['difference'][(df['unit'] == neuron) & (df['dim'] == 0)], percentiles[0]) for neuron in np.arange(0, num_neurons)])
            rw_two_neg = np.array([rw_two_diff[neuron] < np.percentile(df['difference'][(df['unit'] == neuron) & (df['dim'] == 1)], percentiles[0]) for neuron in np.arange(0, num_neurons)])
            ## Assign boolean arrays saying whether that unit_id is pos/neg modulated by reward
            C = C.assign_coords(rw_one_pos=('unit_id', rw_one_pos),
                                rw_two_pos=('unit_id', rw_two_pos),
                                rw_one_neg=('unit_id', rw_one_neg),
                                rw_two_neg=('unit_id', rw_two_neg))
            ## Save data
            if not os.path.exists(save_path):
                os.makedirs(save_path)
            C.to_netcdf(pjoin(save_path, f'{session}'))

## Close cluster after entire loop      
client.close()
cluster.close()

In [ ]:
client.close()
cluster.close()